https://www.kaggle.com/code/hothothot/pz4-nlp

In [31]:
import pickle
import pandas as pd
from transformers import pipeline
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [2]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-conversational')
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-conversational')

tokenizer, model

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(BertTokenizerFast(name_or_path='DeepPavlov/rubert-base-cased-conversational', vocab_size=100792, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 },
 BertModel(
   (embeddings): BertEmbeddings(
     (word_embeddin

In [71]:
device = 'cuda'

In [72]:
model.to(device);

In [73]:
df = pd.read_pickle('/kaggle/input/vacancies-2k/df_vacancies_2k.pkl')
df.head(3)

,id,description,salary
0,88979575,"[finstar, financial, group, крупный, междунаро...",178780.0
1,91380259,"[adgo, это, молодой, динамично, развивающийся,...",150000.0
2,90633183,"[предстоять, заниматься, поддержка, работоспос...",250000.0


In [74]:
x = df['description'].apply(' '.join).values.tolist()
y = df['salary'].values.tolist()

x[:1], y[:1]

(['finstar financial group крупный международный холдинг который входить компания весь мир основной бизнес онлайн кредитование банковский сегмент амбициозный дружный активный команда который работать современный финансовый сервис фокусироваться проект страна юго восточный азия связь динамичный развитие искать команда новый bi analyst релокация вьетнам предстоять заниматься взаимодействие бизнес заказчик вопрос отчётность анализ требование разработка витрина данные участие проектирование развитие архитектура dwh разработка отчётность power bi автоматизация бизнес процесс написание различный сложность sql скрипт оптимизация sql скрипт возможно использование python цель автоматизация организация etl процесс подойти вакансия математический технический высокий образование опыт работа power bi иной bi инструмент понимание отличаться оптимальный sql запрос неоптимальный опыт разработка сложный sql скрипт процедура разговорный английский язык b2 выше готовность релокация г хошимин вьетнам пред

In [97]:
def get_features(x, device):
    tokenized = tokenizer(x, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
    with torch.no_grad():
        return model(**tokenized)[0][:, 0]

batch_size = 128
X = get_features(x[:1], device)

for i in tqdm(range(1, len(x), batch_size)):
    _x = x[i:i + batch_size]
    _X = get_features(_x, device)
    X = torch.concat([X, _X])
    
X.shape

100%|██████████| 16/16 [00:33<00:00,  2.09s/it]


torch.Size([2000, 768])

In [98]:
X = X.to('cpu')

train_X, test_X = X[:1900], X[1900:]
train_Y, test_Y = y[:1900], y[1900:]

In [172]:
class Metrics():

    def __call__(self, y_test: np.ndarray, y_pred: np.ndarray) -> None:
        self.y_test = np.array(y_test)  # лень переделывать
        self.y_pred = np.array(y_pred)
        print(
            f'MAE:\t{self.MAE()}',
            f'MSE:\t{self.MSE()}',
            f'RMSE:\t{self.RMSE()}',
            f'MAPE:\t{self.MAPE()}',
            f'R^2:\t{self.score()}',
            sep='\n'
        )

    def MAE(self) -> float:
        res = 1/len(self.y_pred) * np.sum(abs(self.y_test-self.y_pred))
        return res

    def MSE(self) -> float:
        res = 1/len(self.y_pred) * np.sum((self.y_test-self.y_pred)**2)
        return res

    def RMSE(self) -> float:
        res = np.sqrt(self.MSE())
        return res

    def MAPE(self) -> float:
        res = 1/len(self.y_pred) * np.sum(abs((self.y_test-self.y_pred)/self.y_test))
        return res

    def score(self):
        res = 1 - self.MSE() * len(self.y_pred) / np.sum((self.y_test - np.mean(self.y_test))**2)
        return res

calc_metrics = Metrics()

In [150]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [123]:
params = {
    'alpha': [0.5, 1, 2, 4],
    'warm_start': [True, False],
}

lasso = GridSearchCV(Lasso(), params).fit(train_X, train_Y)

In [131]:
lasso.best_estimator_.coef_

array([-0.00000000e+00,  3.39174985e+04,  1.26011569e+04,  9.28816905e+04,
       -4.10452548e+04,  3.06436062e+03,  2.61990346e+04, -3.42421831e+04,
        1.22858100e+04, -5.57066451e+04,  1.05570489e+05,  1.14218444e+05,
        3.79389688e+04,  1.00368377e+05,  1.42592514e+04, -5.11343540e+03,
       -1.95609405e+05, -1.47429450e+05,  3.12428461e+04,  3.33327174e+03,
        8.84593521e+04, -6.06834009e+04,  6.87290925e+02,  1.09529240e+05,
        5.12619622e+04,  4.54995495e+04, -2.50587000e+04,  2.78536848e+04,
        1.98013529e+04,  7.58376193e+04, -3.47718707e+03, -8.21128055e+04,
        3.91507921e+04, -7.80724273e+04, -1.01668598e+05, -3.94853960e+04,
        2.22296594e+04,  3.01094430e+04,  7.36539524e+04,  2.44617820e+04,
       -1.26693126e+05, -8.87191058e+04, -6.73287350e+03,  3.61932649e+04,
       -9.75191066e+01, -8.62963356e+04,  7.95905583e+03, -8.83794946e+03,
       -1.99359713e+04, -4.55508615e+03,  1.18041310e+05,  1.13491343e+05,
       -6.16473359e+04,  

In [132]:
pred = lasso.predict(test_X)
pred

array([211187.02024868, 266016.35731277, 186655.35894604,  23552.30654473,
       143280.27939836, 138512.91654252, 149913.7747054 , 380606.92034688,
       316755.71321975, 333486.7452985 ,  88176.48438502, 215918.41095098,
       282189.93981696, 144130.96046074, 211950.90677008, 404940.67069965,
       152655.65581939, 220884.06208692, 223531.78958213, 372238.43515416,
       272503.80322157, 191069.82057788, 208247.1687617 , 375359.25146919,
       203147.69030041, 200137.73666491, 204653.78232016, 326405.78214685,
       298827.08846681,  82647.99802059, 255642.55621782,  64517.0057447 ,
       254631.14783823, 325482.45356008, 103024.27570292, 381257.9047282 ,
       -24043.32016879,  71561.20085037, 220778.90127507, 304059.32136619,
       151866.67408854, 199792.68641205, 269010.86260083, 124921.15871388,
       133929.58259205, 130774.462256  , 169495.03619686, 233469.01966896,
       156432.27006697,  90136.02553484, 190201.33850887, 101910.01351485,
       213867.42998595, 2

In [173]:
calc_metrics(test_Y, pred)

MAE:	72413.48116009962
MSE:	10931933583.215492
RMSE:	104555.88736754852
MAPE:	0.6994094004276302
R^2:	-0.2098953415341347


In [135]:
params = {
    'max_depth': range(3, 16, 2),
}

dtr = GridSearchCV(DecisionTreeRegressor(), params).fit(train_X, train_Y)
dtr

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': range(3, 16, 2)})

In [137]:
dtr.best_params_

{'max_depth': 3}

In [174]:
calc_metrics(test_Y, dtr.predict(test_X))

MAE:	73076.15938132681
MSE:	8303831434.2099085
RMSE:	91125.36109234305
MAPE:	0.6423524994448374
R^2:	0.0809707273962279


In [139]:
from catboost import CatBoostRegressor

In [143]:
cbr = CatBoostRegressor(verbose=False).fit(np.array(train_X), train_Y)

In [175]:
calc_metrics(test_Y, cbr.predict(np.array(test_X)))

MAE:	55473.32378995458
MSE:	5925346653.331136
RMSE:	76976.27331412671
MAPE:	0.4522350375610095
R^2:	0.3442103120853822


In [151]:
from sklearn.decomposition import PCA

In [152]:
train_X.shape

torch.Size([1900, 768])

In [160]:
pca = PCA(n_components=128)
train_X_pca = pca.fit_transform(train_X, train_Y)
test_X_pca = pca.transform(test_X)

np.sum(pca.explained_variance_ratio_)

0.9519685906755454

In [176]:
cbr_pca = CatBoostRegressor(verbose=False).fit(np.array(train_X_pca), train_Y)

calc_metrics(test_Y, cbr_pca.predict(np.array(test_X_pca)))

MAE:	55924.53516727979
MSE:	5880683893.266672
RMSE:	76685.61725165074
MAPE:	0.46742598031985944
R^2:	0.34915337773161137


In [177]:
lasso_pca = Lasso().fit(np.array(train_X_pca), train_Y)
calc_metrics(test_Y, lasso_pca.predict(np.array(test_X_pca)))

MAE:	62659.73498432756
MSE:	6825382798.426732
RMSE:	82615.87497827964
MAPE:	0.549104522563951
R^2:	0.24459851597682936


In [163]:
params = {
    'max_depth': range(3, 13, 2),
}
dtr_pca = GridSearchCV(DecisionTreeRegressor(), params).fit(np.array(train_X_pca), train_Y)

In [178]:
calc_metrics(test_Y, dtr_pca.predict(np.array(test_X_pca)))

MAE:	68142.36607237956
MSE:	7567814148.982459
RMSE:	86993.1844973068
MAPE:	0.5799030694812425
R^2:	0.16242968229260057
